In [1]:
## Import Library
import openpyxl
import collections
import os
from PIL import Image
import random
import json
from shutil import copyfile

import argparse
import logging
import numpy as np
import torch
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm

import utils
import model.data_loader as data_loader
from evaluate import evaluate
import loss_and_metrics

# Recognizing Diseased Coffee Leaves Using Deep Learning


In this project, given a set of images of coffee leaves, this project will explore deep learning algorithms (both fully connected and convolution
neural networks) that output the correct labels for the conditions of the coffee leaves. 

The six conditions are 
* Healthy (H)
* Rust Level 1 (RL1)
* Rust Level 2 (RL2)
* Rust Level 3 (RL3)
* Rust Level 4 (RL4)
* Red Spider Mites (RSM)

For this project, we explore three different tasks:
1) Given the full dataset, classify them into the 6 categories mentioned above.
2) Given the full dataset, classify them into 3 categories (H, RL, RSM).
3) Given the images from the healthy and rust level categories only, classify them into 5 categories (H, RL1, RL2, RL3, RL4) using a regression-based approach.

## Dataset
**Robusta dataset**: [Dataset](https://drive.google.com/drive/folders/13fFAQHU_-Ar0zg6RHl1FTLOE3I2QnCWI?usp=sharing)


### Setting the Virtual Environment and Installing Requirements
Requirements:
Run the follow commands:
```sh
$ pip install -r code/requirements.txt
```
Processing the Dataset
After downloading the annotations and images, they should be placed inside the CoffeeLeafNoteBook directory as follows

CoffeeLeafNoteBook/Annotations/{annotation files}
CoffeeLeafNoteBook/Photos/{.jpg files}

### To process the images for Task 1 above, run the following command:

* [Data Processing](#dataprocessing)

### To process the images for Task 2 above, run the following command:
* [Data Processing For Three Class](#dataprocessingthreeclass)

### To process the images for Task 3 above, run the following command:
* [Data Processing For RegressionTask](#dataprocessingregression)

### Training models
To train models, first create a ```params.json``` file inside the ```experiments/{A}/{B}``` directory, where
* {A} is either ```six_classes```, ```three_classes```, or ```regression```
* {B} is the descriptive name for the experiment model

**Then for Task 1 and 2, we use [Training the model](#trainmodel)**

**For Task 3, we use the [Train Regression](#trainmodelregression)**

### Evaluate on a Saved Model
**To evaluate on a saved model, run [Evaluate model](#evaluatef1)**


By default, it will evaluate on only the training and validation set.

+ To evaluate on the test set, the --testSet True flag must be added.
+ To not evaluate on the training and validation set, we can set the --trainAndVal False.
+ To evaluate only on the test set, we can set both flags --trainAndVal False --testSet True



## Data Processing <a class="anchor" id="dataprocessing"></a>
This is code for data processing task

**Given the full dataset, classify them into the 6 categories mentioned above.**

In [ ]:

IMG_DIM = 720
xlsx_path = "./Annotations/RoCoLe-classes.xlsx"
annotation_json_path = "./Annotations/RoCoLe-json.json"
photo_path_prefix = "./Photos/"
binary_path = "./binary/"
multiclass_path = "./multiclass/"

binary_classifications = {
    "healthy": 0,
    "unhealthy": 1
}

multiclass_classifications = {
    "healthy": 0,
    "rust_level_1": 1,
    "rust_level_2": 2,
    "rust_level_3": 3,
    "rust_level_4": 4,
    "red_spider_mite": 5
}

def split_into_train_val_test(dict):
    random.seed(230)

    test = []
    val = []
    train = []

    for category in dict:
        img_names = list(dict[category])
        img_names.sort()
        random.shuffle(img_names)

        test_split = int(0.1 * len(img_names))
        val_split = int(.18 * len(img_names))

        test_img_names = img_names[:test_split]
        val_img_names = img_names[test_split: test_split + val_split]
        train_img_names = img_names[test_split + val_split:]

        test.extend(test_img_names)
        val.extend(val_img_names)
        train.extend(train_img_names)

    return {
        "test": set(test),
        "val": set(val),
        "train": set(train)
    }


def generate_binary_and_multiclass_dict_old():
    wb_obj = openpyxl.load_workbook(xlsx_path)
    sheet_obj = wb_obj.active

    binary_dict = collections.defaultdict(set)
    multiclass_dict = collections.defaultdict(set)

    num_row = sheet_obj.max_row

    for i in range(2, num_row + 1):
        image_name = sheet_obj.cell(row=i, column=1).value
        binary = sheet_obj.cell(row=i, column=2).value
        multiclass = sheet_obj.cell(row=i, column=3).value

        binary_dict[binary].add(image_name)
        multiclass_dict[multiclass].add(image_name)

    return binary_dict, multiclass_dict


def generate_train_val_test_split(binary_dict, multiclass_dict):
    binary_split = split_into_train_val_test(binary_dict)
    multiclass_split = split_into_train_val_test(multiclass_dict)
    return binary_split, multiclass_split


def get_split(img, split_dict):
    if img in split_dict["test"]:
        return "test"
    if img in split_dict["val"]:
        return "val"
    return "train"


def resize_and_save(filename, output_path, size=IMG_DIM):
    """Resize the image contained in `filename` and save it to the `output_dir`"""
    image = Image.open(filename)
    # Use bilinear interpolation instead of the default "nearest neighbor" method
    image = image.resize((size, size), Image.BILINEAR)
    image.save(output_path)


def copy_photo_files_into_directories(classification_dict, new_classification_path, classification_type, split_dict):
    binary_or_multi = "binary" if "binary" in new_classification_path else "multiclass"
    for (category, images) in classification_dict.items():
        num = str(classification_type[category])
        for img in images:
            split = get_split(img, split_dict)
            make_dir(os.path.join("just_splitted", binary_or_multi, split))
            new_img_path = os.path.join("just_splitted", binary_or_multi, split, num + "_" + img)
            resize_and_save(os.path.join(photo_path_prefix, img), new_img_path)
            # copyfile(os.path.join("just_splitted", "cropped", img), new_img_path)


def categorize_train_val_test_split(verbose = False):
    (binary_dict, multiclass_dict) = generate_binary_and_multiclass_dict_old()
    if verbose:
        print("Finished categorizing pictures into their respective classes for binary and multiclass classification")
    binary_split, multiclass_split = generate_train_val_test_split(binary_dict, multiclass_dict)
    if verbose:
        print("Finished splitting dataset")
    # copy_photo_files_into_directories(binary_dict, binary_path, binary_classifications, binary_split)
    # if verbose:
    #     print("Finished copying photos into the 'binary' folder")
    copy_photo_files_into_directories(multiclass_dict, multiclass_path, multiclass_classifications, multiclass_split)
    if verbose:
        print("Finished copying photos into the 'multiclass' folder")


def make_dir(path):
    path = os.path.abspath(os.path.join(path))

    if not os.path.exists(path):
        try:
            os.makedirs(path)
        except Exception as e:
            # Raise if directory can't be made, because image cuts won't be saved.
            print('Error creating directory')
            raise e

def generate_binary_and_multiclass_dict(img_dimension = IMG_DIM):
    binary_dict = collections.defaultdict(set)
    multiclass_dict = collections.defaultdict(set)

    make_dir(os.path.join("zoom_cropped_and_splitted", "cropped"))
    with open(annotation_json_path) as json_file:
        data = json.load(json_file)
        ct = 0
        for pic_annotation in data:
            if ct % 25 == 0: print(ct)
            leaf_obj = pic_annotation["Label"]["Leaf"][0]
            geometry = leaf_obj["geometry"]
            img_name = pic_annotation["External ID"]

            binary_classif = leaf_obj["state"]
            multi_classif = pic_annotation["Label"]["classification"]
            classif_num = multiclass_classifications[multi_classif]

            image = Image.open(os.path.join(photo_path_prefix, img_name))
            width, height = image.size
            midx = int(width/2)
            midy = int(height/2)
            img_dimension_temp = img_dimension * 2
            zoom_cropped_img = image.crop((midx - img_dimension_temp, midy - img_dimension_temp, midx + img_dimension_temp, midy + img_dimension_temp))
            zoom_cropped_img_name = str(classif_num) + "_" + img_name
            zoom_cropped_img.save(os.path.join("zoom_cropped_and_splitted", "cropped", zoom_cropped_img_name))
            binary_dict[binary_classif].add(zoom_cropped_img_name)
            multiclass_dict[multi_classif].add(zoom_cropped_img_name)

            # for i in range(len(geometry)):
            #     xy = geometry[i]
            #
            #     x = xy["x"]
            #     y = xy["y"]
            #     xmin = x - img_dimension
            #     xmax = x + img_dimension
            #     ymin = y - img_dimension
            #     ymax = y + img_dimension
            #     if xmin < 0 or ymin < 0 or xmax > width or ymax > height:
            #         continue
            #
            #     new_img = image.crop((xmin, ymin, xmax, ymax))
            #     new_img_name = str(classif_num) + "_" + "{}_".format(i) + img_name
            #     new_img.save(os.path.join("cropped", new_img_name))
            #
            #     binary_dict[binary_classif].add(new_img_name)
            #     multiclass_dict[multi_classif].add(new_img_name)
            ct += 1

    return binary_dict, multiclass_dict

def main():
    categorize_train_val_test_split(True)


if __name__ == "__main__":
    main()

## Data Processing For Three Class <a class="anchor" id="dataprocessingthreeclass"></a>
This is code for data processing for three class

**Given the full dataset, classify them into 3 categories (H, RL, RSM).**

In [ ]:
IMG_DIM = 720
xlsx_path = "./Annotations/RoCoLe-classes.xlsx"
photo_path_prefix = "./Photos/"


multiclass_classifications = {
    "healthy": 0,
    "rust_level_1": 1,
    "rust_level_2": 1,
    "rust_level_3": 1,
    "rust_level_4": 1,
    "red_spider_mite": 2
}

def split_into_train_val_test(dict):
    random.seed(230)

    test = []
    val = []
    train = []

    for category in dict:
        img_names = list(dict[category])
        img_names.sort()
        random.shuffle(img_names)

        test_split = int(0.1 * len(img_names))
        val_split = int(.18 * len(img_names))

        test_img_names = img_names[:test_split]
        val_img_names = img_names[test_split: test_split + val_split]
        train_img_names = img_names[test_split + val_split:]

        test.extend(test_img_names)
        val.extend(val_img_names)
        train.extend(train_img_names)

    return {
        "test": set(test),
        "val": set(val),
        "train": set(train)
    }


def generate_binary_and_multiclass_dict():
    wb_obj = openpyxl.load_workbook(xlsx_path)
    sheet_obj = wb_obj.active

    multiclass_dict = collections.defaultdict(set)

    num_row = sheet_obj.max_row

    for i in range(2, num_row + 1):
        image_name = sheet_obj.cell(row=i, column=1).value
        binary = sheet_obj.cell(row=i, column=2).value
        multiclass = sheet_obj.cell(row=i, column=3).value

        multiclass_dict[multiclass].add(image_name)

    return multiclass_dict


def generate_train_val_test_split(multiclass_dict):
    multiclass_split = split_into_train_val_test(multiclass_dict)
    return multiclass_split


def get_split(img, split_dict):
    if img in split_dict["test"]:
        return "test"
    if img in split_dict["val"]:
        return "val"
    return "train"


def resize_and_save(filename, output_path, size=IMG_DIM):
    """Resize the image contained in `filename` and save it to the `output_dir`"""
    image = Image.open(filename)
    # Use bilinear interpolation instead of the default "nearest neighbor" method
    image = image.resize((size, size), Image.BILINEAR)
    image.save(output_path)


def copy_photo_files_into_directories(classification_dict, classification_type, split_dict):
    for (category, images) in classification_dict.items():
        num = str(classification_type[category])
        for img in images:
            split = get_split(img, split_dict)
            make_dir(os.path.join("three_classes", "multiclass", split))
            new_img_path = os.path.join("three_classes", "multiclass", split, num + "_" + img)
            resize_and_save(os.path.join(photo_path_prefix, img), new_img_path)


def categorize_train_val_test_split(verbose = False):
    multiclass_dict = generate_binary_and_multiclass_dict()
    if verbose:
        print("Finished categorizing pictures into their respective classes for multiclass classification")
    multiclass_split = generate_train_val_test_split(multiclass_dict)
    if verbose:
        print("Finished splitting dataset")
    copy_photo_files_into_directories(multiclass_dict, multiclass_classifications, multiclass_split)
    if verbose:
        print("Finished copying photos into the 'multiclass' folder")


def make_dir(path):
    path = os.path.abspath(os.path.join(path))

    if not os.path.exists(path):
        try:
            os.makedirs(path)
        except Exception as e:
            # Raise if directory can't be made, because image cuts won't be saved.
            print('Error creating directory')
            raise e

def main():
    categorize_train_val_test_split(True)


if __name__ == "__main__":
    main()

## Data Processing For Regression Task <a class="anchor" id="dataprocessingregression"></a>
This is code for data processing for Regression Task

**Given the images from the healthy and rust level categories only, classify them into 5 categories (H, RL1, RL2, RL3, RL4) using a regression-based approach.**


In [ ]:
IMG_DIM = 720
xlsx_path = "./Annotations/RoCoLe-classes.xlsx"
photo_path_prefix = "./Photos/"


multiclass_classifications = {
    "healthy": 0,
    "rust_level_1": 1,
    "rust_level_2": 2,
    "rust_level_3": 3,
    "rust_level_4": 4,
    "red_spider_mite": 5
}

def split_into_train_val_test(dict):
    random.seed(230)

    test = []
    val = []
    train = []

    for category in dict:
        img_names = list(dict[category])
        img_names.sort()
        random.shuffle(img_names)

        test_split = int(0.1 * len(img_names))
        val_split = int(.18 * len(img_names))

        test_img_names = img_names[:test_split]
        val_img_names = img_names[test_split: test_split + val_split]
        train_img_names = img_names[test_split + val_split:]

        test.extend(test_img_names)
        val.extend(val_img_names)
        train.extend(train_img_names)

    return {
        "test": set(test),
        "val": set(val),
        "train": set(train)
    }


def generate_binary_and_multiclass_dict():
    wb_obj = openpyxl.load_workbook(xlsx_path)
    sheet_obj = wb_obj.active

    multiclass_dict = collections.defaultdict(set)

    num_row = sheet_obj.max_row

    for i in range(2, num_row + 1):
        image_name = sheet_obj.cell(row=i, column=1).value
        multiclass = sheet_obj.cell(row=i, column=3).value

        multiclass_dict[multiclass].add(image_name)

    return multiclass_dict


def generate_train_val_test_split(multiclass_dict):
    multiclass_split = split_into_train_val_test(multiclass_dict)
    return multiclass_split


def get_split(img, split_dict):
    if img in split_dict["test"]:
        return "test"
    if img in split_dict["val"]:
        return "val"
    return "train"


def resize_and_save(filename, output_path, size=IMG_DIM):
    """Resize the image contained in `filename` and save it to the `output_dir`"""
    image = Image.open(filename)
    # Use bilinear interpolation instead of the default "nearest neighbor" method
    image = image.resize((size, size), Image.BILINEAR)
    image.save(output_path)


def copy_photo_files_into_directories(classification_dict, classification_type, split_dict):
    for (category, images) in classification_dict.items():
        num = classification_type[category]
        if num == 5: continue
        num_str = str(num)
        for img in images:
            split = get_split(img, split_dict)
            make_dir(os.path.join("regression", "multiclass", split))
            new_img_path = os.path.join("regression", "multiclass", split, num_str + "_" + img)
            resize_and_save(os.path.join(photo_path_prefix, img), new_img_path)


def categorize_train_val_test_split(verbose = False):
    multiclass_dict = generate_binary_and_multiclass_dict()
    if verbose:
        print("Finished categorizing pictures into their respective classes for multiclass classification")
    multiclass_split = generate_train_val_test_split(multiclass_dict)
    if verbose:
        print("Finished splitting dataset")
    copy_photo_files_into_directories(multiclass_dict, multiclass_classifications, multiclass_split)
    if verbose:
        print("Finished copying photos into the 'multiclass' folder")


def make_dir(path):
    path = os.path.abspath(os.path.join(path))

    if not os.path.exists(path):
        try:
            os.makedirs(path)
        except Exception as e:
            # Raise if directory can't be made, because image cuts won't be saved.
            print('Error creating directory')
            raise e

def main():
    categorize_train_val_test_split(True)


if __name__ == "__main__":
    main()

## Train Model For Task 1 and 2 <a class="anchor" id="trainmodel"></a>
This is code for training the model for task 1 and task 2

+ Task 1: Given the full dataset, classify them into the 6 categories mentioned above.
+ Task 2: Given the full dataset, classify them into 3 categories (H, RL, RSM).

In [ ]:
"""Train the model"""

import argparse
import logging
import os
import numpy as np
import torch
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm

import utils
import model.data_loader as data_loader
from evaluate import evaluate
import loss_and_metrics

parser = argparse.ArgumentParser()
parser.add_argument('--model_dir', default='experiments/six_classes/example_trans_learning',
                    help="Directory containing params.json")
parser.add_argument('--restore_file', default=None,
                    help="Optional, name of the file in --model_dir containing weights to reload before \
                    training")  # 'best' or 'train'


def train(model, optimizer, loss_fn, dataloader, metrics, params):
    """Train the model on `num_steps` batches

    Args:
        model: (torch.nn.Module) the neural network
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches training data
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to training mode
    model.train()

    # summary for current training loop and a running average object for loss
    summ = []
    loss_avg = utils.RunningAverage()

    # Use tqdm for progress bar
    with tqdm(total=len(dataloader)) as t:
        for i, (train_batch, labels_batch) in enumerate(dataloader):
            # move to GPU if available
            if params.cuda:
                train_batch, labels_batch = train_batch.cuda(
                    non_blocking=True), labels_batch.cuda(non_blocking=True)
            # convert to torch Variables
            train_batch, labels_batch = Variable(
                train_batch), Variable(labels_batch)

            # compute model output and loss
            output_batch = model(train_batch)
            loss = loss_fn(output_batch, labels_batch)

            # clear previous gradients, compute gradients of all variables wrt loss
            optimizer.zero_grad()
            loss.backward()

            # performs updates using calculated gradients
            optimizer.step()

            # Evaluate summaries only once in a while
            if i % params.save_summary_steps == 0:
                # extract data from torch Variable, move to cpu, convert to numpy arrays
                output_batch = output_batch.data.cpu().numpy()
                labels_batch = labels_batch.data.cpu().numpy()

                # compute all metrics on this batch
                summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                                 for metric in metrics}
                summary_batch['loss'] = loss.item()
                summ.append(summary_batch)

            # update the average loss
            loss_avg.update(loss.item())

            t.set_postfix(loss='{:05.3f}'.format(loss_avg()))
            t.update()

    # compute mean of all metrics in summary
    metrics_mean = {metric: np.mean([x[metric]
                                     for x in summ]) for metric in summ[0]}
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v)
                                for k, v in metrics_mean.items())
    logging.info("- Train metrics: " + metrics_string)


def train_and_evaluate(model, train_dataloader, val_dataloader, optimizer, loss_fn, metrics, params, model_dir,
                       restore_file=None):
    """Train the model and evaluate every epoch.

    Args:
        model: (torch.nn.Module) the neural network
        train_dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches training data
        val_dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches validation data
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        model_dir: (string) directory containing config, weights and log
        restore_file: (string) optional- name of file to restore from (without its extension .pth.tar)
    """
    # reload weights from restore_file if specified
    if restore_file is not None:
        restore_path = os.path.join(
            args.model_dir, args.restore_file + '.pth.tar')
        logging.info("Restoring parameters from {}".format(restore_path))
        utils.load_checkpoint(restore_path, model, optimizer)

    best_val_macro_f1 = 0.0

    for epoch in range(params.num_epochs):
        # Run one epoch
        logging.info("Epoch {}/{}".format(epoch + 1, params.num_epochs))

        # compute number of batches in one epoch (one full pass over the training set)
        train(model, optimizer, loss_fn, train_dataloader, metrics, params)

        # Evaluate for one epoch on validation set
        val_metrics = evaluate(model, loss_fn, val_dataloader, metrics, params)

        val_macro_f1 = val_metrics['macro f1']
        is_best = val_macro_f1 >= best_val_macro_f1

        # Save weights
        utils.save_checkpoint({'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optim_dict': optimizer.state_dict()},
                              is_best=is_best,
                              checkpoint=model_dir)

        # If best_eval, best_save_path
        if is_best:
            logging.info("- Found new best macro f1")
            best_val_macro_f1 = val_macro_f1

            # Save best val metrics in a json file in the model directory
            best_json_path = os.path.join(
                model_dir, "metrics_val_best_weights.json")
            utils.save_dict_to_json(val_metrics, best_json_path)

        # Save latest val metrics in a json file in the model directory
        last_json_path = os.path.join(
            model_dir, "metrics_val_last_weights.json")
        utils.save_dict_to_json(val_metrics, last_json_path)


if __name__ == '__main__':
    # Load the parameters from json file
    args = parser.parse_args()
    json_path = os.path.join(args.model_dir, "params.json")
    assert os.path.isfile(
        json_path), "No json configuration file found at {}".format(json_path)
    params = utils.Params(json_path)

    # use GPU if available
    params.cuda = torch.cuda.is_available()
    data_dir = 'just_splitted/multiclass' if 'six_classes' in args.model_dir else 'three_classes/multiclass'
    # Set the random seed for reproducible experiments
    torch.manual_seed(230)
    if params.cuda:
        torch.cuda.manual_seed(230)

    # Set the logger
    utils.set_logger(os.path.join(args.model_dir, 'train.log'))

    # Create the input data pipeline
    logging.info("Loading the datasets...")

    # fetch dataloaders
    dataloaders = data_loader.fetch_dataloader(
        ['train', 'val'], data_dir, params)
    train_dl = dataloaders['train']
    val_dl = dataloaders['val']

    logging.info("- done.")

    # model selected is based on params.net
    model = utils.get_desired_model(params)

    w_decay = params.weight_decay if hasattr(params, 'weight_decay') else 0.0
    optimizer = optim.Adam(model.parameters(), lr=params.learning_rate, weight_decay=w_decay)

    # fetch loss function and metrics
    loss_fn = loss_and_metrics.loss_fn
    metrics = loss_and_metrics.metrics

    # Train the model
    logging.info("Starting training for {} epoch(s)".format(params.num_epochs))
    train_and_evaluate(model, train_dl, val_dl, optimizer, loss_fn, metrics, params, args.model_dir,
                       args.restore_file)


## Train Model For Task 3 <a class="anchor" id="trainmodelregression"></a>
This is code training the model for task 3 

**Task 3: Given the images from the healthy and rust level categories only, classify them into 5 categories (H, RL1, RL2, RL3, RL4) using a regression-based approach.**

In [ ]:
"""Train the model"""

import argparse
import logging
import os
import numpy as np
import torch
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm

import utils
import model.regression_adopted_cnn as regression_cnn
import model.data_loader as data_loader
from evaluate import evaluate
import regression_loss_and_metrics

parser = argparse.ArgumentParser()
parser.add_argument('--model_dir',
                    help="Directory containing params.json")
parser.add_argument('--restore_file', default=None,
                    help="Optional, name of the file in --model_dir containing weights to reload before \
                    training")  # 'best' or 'train'

def get_desired_model(params):
    return regression_cnn.Regression_Adopted_NN(params).cuda() if params.cuda else regression_cnn.Regression_Adopted_NN(params)

def train(model, optimizer, loss_fn, dataloader, metrics, params):
    """Train the model on `num_steps` batches

    Args:
        model: (torch.nn.Module) the neural network
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches training data
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to training mode
    model.train()

    # summary for current training loop and a running average object for loss
    summ = []
    loss_avg = utils.RunningAverage()

    # Use tqdm for progress bar
    with tqdm(total=len(dataloader)) as t:
        for i, (train_batch, labels_batch) in enumerate(dataloader):
            # move to GPU if available
            if params.cuda:
                train_batch, labels_batch = train_batch.cuda(
                    non_blocking=True), labels_batch.cuda(non_blocking=True)
            # convert to torch Variables
            train_batch, labels_batch = Variable(
                train_batch), Variable(labels_batch)

            # compute model output and loss
            output_batch = model(train_batch)
            loss = loss_fn(output_batch, labels_batch)

            # clear previous gradients, compute gradients of all variables wrt loss
            optimizer.zero_grad()
            loss.backward()

            # performs updates using calculated gradients
            optimizer.step()

            # Evaluate summaries only once in a while
            if i % params.save_summary_steps == 0:
                # extract data from torch Variable, move to cpu, convert to numpy arrays
                output_batch = output_batch.data.cpu().numpy()
                labels_batch = labels_batch.data.cpu().numpy()

                # compute all metrics on this batch
                summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                                 for metric in metrics}
                summary_batch['loss'] = loss.item()
                summ.append(summary_batch)

            # update the average loss
            loss_avg.update(loss.item())

            t.set_postfix(loss='{:05.3f}'.format(loss_avg()))
            t.update()

    # compute mean of all metrics in summary
    metrics_mean = {metric: np.mean([x[metric]
                                     for x in summ]) for metric in summ[0]}
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v)
                                for k, v in metrics_mean.items())
    logging.info("- Train metrics: " + metrics_string)


def train_and_evaluate(model, train_dataloader, val_dataloader, optimizer, loss_fn, metrics, params, model_dir,
                       restore_file=None):
    """Train the model and evaluate every epoch.

    Args:
        model: (torch.nn.Module) the neural network
        train_dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches training data
        val_dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches validation data
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        model_dir: (string) directory containing config, weights and log
        restore_file: (string) optional- name of file to restore from (without its extension .pth.tar)
    """
    # reload weights from restore_file if specified
    if restore_file is not None:
        restore_path = os.path.join(
            args.model_dir, args.restore_file + '.pth.tar')
        logging.info("Restoring parameters from {}".format(restore_path))
        utils.load_checkpoint(restore_path, model, optimizer)

    best_val_macro_f1 = 0.0

    for epoch in range(params.num_epochs):
        # Run one epoch
        logging.info("Epoch {}/{}".format(epoch + 1, params.num_epochs))

        # compute number of batches in one epoch (one full pass over the training set)
        train(model, optimizer, loss_fn, train_dataloader, metrics, params)

        # Evaluate for one epoch on validation set
        val_metrics = evaluate(model, loss_fn, val_dataloader, metrics, params)

        val_macro_f1 = val_metrics['macro f1']
        is_best = val_macro_f1 >= best_val_macro_f1

        # Save weights
        utils.save_checkpoint({'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optim_dict': optimizer.state_dict()},
                              is_best=is_best,
                              checkpoint=model_dir)

        # If best_eval, best_save_path
        if is_best:
            logging.info("- Found new best macro f1")
            best_val_macro_f1 = val_macro_f1

            # Save best val metrics in a json file in the model directory
            best_json_path = os.path.join(
                model_dir, "metrics_val_best_weights.json")
            utils.save_dict_to_json(val_metrics, best_json_path)

        # Save latest val metrics in a json file in the model directory
        last_json_path = os.path.join(
            model_dir, "metrics_val_last_weights.json")
        utils.save_dict_to_json(val_metrics, last_json_path)


if __name__ == '__main__':
    # Load the parameters from json file
    args = parser.parse_args()
    json_path = os.path.join(args.model_dir, "params.json")
    assert os.path.isfile(
        json_path), "No json configuration file found at {}".format(json_path)
    params = utils.Params(json_path)

    # use GPU if available
    params.cuda = torch.cuda.is_available()

    # Set the random seed for reproducible experiments
    torch.manual_seed(230)
    if params.cuda:
        torch.cuda.manual_seed(230)

    # Set the logger
    utils.set_logger(os.path.join(args.model_dir, 'train.log'))

    # Create the input data pipeline
    logging.info("Loading the datasets...")

    # fetch dataloaders
    data_dir = 'regression/multiclass'
    dataloaders = data_loader.fetch_dataloader(
        ['train', 'val'], data_dir, params)
    train_dl = dataloaders['train']
    val_dl = dataloaders['val']

    logging.info("- done.")

    # model selected is based on params.net
    model = get_desired_model(params)

    w_decay = params.weight_decay if hasattr(params, 'weight_decay') else 0.0
    optimizer = optim.Adam(model.parameters(), lr=params.learning_rate, weight_decay=w_decay)

    # fetch loss function and metrics
    loss_fn = regression_loss_and_metrics.regression_loss_fn
    metrics = regression_loss_and_metrics.regression_metrics

    # Train the model
    logging.info("Starting training for {} epoch(s)".format(params.num_epochs))
    train_and_evaluate(model, train_dl, val_dl, optimizer, loss_fn, metrics, params, args.model_dir,
                       args.restore_file)


## Evaluated Model <a class="anchor" id="evaluatef1"></a>
This is code evaluated model

**Task 3: Given the images from the healthy and rust level categories only, classify them into 5 categories (H, RL1, RL2, RL3, RL4) using a regression-based approach.**

In [ ]:
"""Evaluates the model"""

import argparse
import logging
import os

import numpy as np
import torch
from torch.autograd import Variable
import utils
import model.data_loader as data_loader
import regression_loss_and_metrics
import loss_and_metrics
import model.regression_adopted_cnn as regression_cnn

parser = argparse.ArgumentParser()
parser.add_argument('--model_dir',
                    help="Directory containing params.json")
parser.add_argument('--restore_file', default='best', help="name of the file in --model_dir \
                     containing weights to load")
parser.add_argument('--testSet', default='False',
                    help="Indicate whether we should get the metrics for the test set.")
parser.add_argument('--trainAndVal', default='True',
                    help="Indicate whether we should get the metrics for the test set.")


def get_model_loss_metrics(args, params):
    if 'regression' in args.model_dir:
        model = regression_cnn.Regression_Adopted_NN(params).cuda() if params.cuda else regression_cnn.Regression_Adopted_NN(params)
        loss_fn = regression_loss_and_metrics.regression_loss_fn
        metrics = regression_loss_and_metrics.regression_metrics
        return model, loss_fn, metrics
    else:
        model = utils.get_desired_model(params)
        loss_fn = loss_and_metrics.loss_fn
        metrics = loss_and_metrics.metrics
        return model, loss_fn, metrics


def compute_and_save_f1(saved_outputs, saved_labels, file):
    conf_matrix, report = utils.f1_metrics(saved_outputs, saved_labels)

    text_file = open(file, "wt")
    text_file.write('Confusion matrix: \n {}\n\n Classification Report: \n {}'.format(conf_matrix, report))
    text_file.close()

def process_output(args, output_batch):
    if 'regression' not in args.model_dir:
        return np.argmax(output_batch, axis=1)
    return np.floor(output_batch + 0.5).flatten()


def evaluate(model, loss_fn, dataloader, metrics, params, which_set, file, args):
    """Evaluate the model on `num_steps` batches.

    Args:
        model: (torch.nn.Module) the neural network
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches data
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to evaluation mode
    model.eval()

    # summary for current eval loop
    summ = []

    saved_outputs = []
    saved_labels = []
    # compute metrics over the dataset
    for data_batch, labels_batch in dataloader:
        # move to GPU if available
        if params.cuda:
            data_batch, labels_batch = data_batch.cuda(
                non_blocking=True), labels_batch.cuda(non_blocking=True)
        # fetch the next evaluation batch
        data_batch, labels_batch = Variable(data_batch), Variable(labels_batch)

        # compute model output
        output_batch = model(data_batch)
        loss = loss_fn(output_batch, labels_batch)

        # extract data from torch Variable, move to cpu, convert to numpy arrays
        output_batch = output_batch.data.cpu().numpy()
        labels_batch = labels_batch.data.cpu().numpy()

        processed_output = process_output(args, output_batch)
        saved_outputs.extend(processed_output)
        saved_labels.extend(labels_batch)

        # compute all metrics on this batch
        summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                         for metric in metrics}
        summary_batch['loss'] = loss.item()
        summ.append(summary_batch)

    # compute mean of all metrics in summary
    metrics_name = {metric: np.mean([x[metric]
                                     for x in summ]) for metric in summ[0]}
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v)
                                for k, v in metrics_name.items())
    logging.info("- {} Metrics : ".format(which_set) + metrics_string)

    compute_and_save_f1(saved_outputs, saved_labels, file)

    return metrics_name


def get_data_dir():
    if 'regression' in args.model_dir:
        return 'regression/multiclass'
    if 'six_classes' in args.model_dir:
        return 'just_splitted/multiclass'
    if 'three_classes' in args.model_dir:
        return 'three_classes/multiclass'

if __name__ == '__main__':
    """
        Evaluate the model on the train, validation, and test set.
    """
    # Load the parameters
    args = parser.parse_args()
    json_path = os.path.join(args.model_dir, 'params.json')
    assert os.path.isfile(
        json_path), "No json configuration file found at {}".format(json_path)
    params = utils.Params(json_path)

    # use GPU if available
    params.cuda = torch.cuda.is_available()     # use GPU is available

    # Set the random seed for reproducible experiments
    torch.manual_seed(230)
    if params.cuda:
        torch.cuda.manual_seed(230)

    # Get the logger
    utils.set_logger(os.path.join(args.model_dir, 'trainAndValidation.log'))

    # Create the input data pipeline
    logging.info("Creating the dataset...")

    # fetch dataloaders
    data_dir = get_data_dir()
    dataloaders = data_loader.fetch_dataloader(['train', 'val', 'test'], data_dir, params)
    train_dl = dataloaders['train']
    val_dl = dataloaders['val']
    test_dl = dataloaders['test']

    logging.info("- done.")

    # Define the model
    model, loss_fn, metrics = get_model_loss_metrics(args, params)

    logging.info("Starting evaluation and calculation of F1 Scores")

    # Reload weights from the saved file
    utils.load_checkpoint(os.path.join(
        args.model_dir, args.restore_file + '.pth.tar'), model)

    # Evaluate Train
    if args.trainAndVal == "True":
        confus_save_path = os.path.join(
            args.model_dir, "confus_f1_train_{}.json".format(args.restore_file))
        train_metrics = evaluate(model, loss_fn, train_dl, metrics, params, 'Train', confus_save_path, args)
        save_path = os.path.join(
            args.model_dir, "metrics_train_{}.json".format(args.restore_file))
        utils.save_dict_to_json(train_metrics, save_path)

        # Evaluate Validation
        confus_save_path = os.path.join(
            args.model_dir, "confus_f1_val_{}.json".format(args.restore_file))
        val_metrics = evaluate(model, loss_fn, val_dl, metrics, params, 'Val', confus_save_path, args)
        save_path = os.path.join(
            args.model_dir, "metrics_val_{}.json".format(args.restore_file))
        utils.save_dict_to_json(val_metrics, save_path)

    if args.testSet == "True":
        confus_save_path = os.path.join(
            args.model_dir, "confus_f1_test_{}.json".format(args.restore_file))
        test_metrics = evaluate(model, loss_fn, test_dl, metrics, params, 'Test', confus_save_path, args)
        save_path = os.path.join(
            args.model_dir, "metrics_test_{}.json".format(args.restore_file))
        utils.save_dict_to_json(test_metrics, save_path)
